In [ ]:
import itertools
from collections import deque, defaultdict

# 1. Degree of each vertex and sorted
def get_sorted_degrees(adj_list):
    degrees = {node: len(neighbors) for node, neighbors in adj_list.items()}  # Create a dictionary with node: degree
    return dict(sorted(degrees.items(), key=lambda x: x[1], reverse=True))  # Sort by degree in descending order

# Test
graph = {'A': ['B', 'C'], 'B': ['A'], 'C': ['A']}
print("Sorted Degrees:", get_sorted_degrees(graph))  # Expected: {'A': 2, 'B': 1, 'C': 1}

# 2. Inter-convert representations
def adj_list_to_edge_list(adj_list):
    edges = set()
    for node in adj_list:
        for neighbor in adj_list[node]:
            if (neighbor, node) not in edges:  # Avoid adding the same edge twice in undirected graphs
                edges.add((node, neighbor)) # Add the edge as a tuple
    return list(edges) # Convert the set to a list and return

def adj_list_to_matrix(adj_list):
    nodes = sorted(adj_list.keys())  # Get all nodes in sorted order for consistent indexing
    index = {node: i for i, node in enumerate(nodes)}  # Create a mapping from node to index
    size = len(nodes)
    matrix = [[0]*size for _ in range(size)]  # Initialize size x size matrix with 0s

    for node, neighbors in adj_list.items():
        for neighbor in neighbors:
            matrix[index[node]][index[neighbor]] = 1   # Set matrix cell to 1 if there is an edge
    return nodes, matrix   # Return the order of nodes and the matrix

def edge_list_to_adj_list(edge_list):
    adj_list = defaultdict(list)
    for u, v in edge_list:
        adj_list[u].append(v)  # Add v to u's list
        adj_list[v].append(u) # Add u to v's list to ensure undirected nature
    return dict(adj_list)

# Test
edge_list = adj_list_to_edge_list(graph)
print("Edge List:", edge_list)
nodes, matrix = adj_list_to_matrix(graph)
print("Adjacency Matrix:", matrix)
print("Back to Adjacency List:", edge_list_to_adj_list(edge_list))

# 3. Check adjacency
def are_adjacent(adj_list, u, v):
    return v in adj_list.get(u, []) # Return True if v is present in u’s adjacency list

# Test
print("Are A and B adjacent?", are_adjacent(graph, 'A', 'B'))  # True
print("Are B and C adjacent?", are_adjacent(graph, 'B', 'C'))  # False

# 4. Check if graph is complete
def is_complete(adj_list):
    n = len(adj_list) # Total number of vertices
    return all(len(neighbors) == n-1 for neighbors in adj_list.values())  # Each node must be connected to all others

# Test
complete_graph = {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']}
print("Is complete?", is_complete(complete_graph))  # True
print("Is complete?", is_complete(graph))           # False

# 5. Check if graph is connected
def is_connected(adj_list):
    start = next(iter(adj_list)) # Start BFS from any one node
    visited = set()
    queue = deque([start])

    while queue:
        node = queue.popleft()
        if node not in visited:
            visited.add(node) # Mark node as visited
            queue.extend([n for n in adj_list[node] if n not in visited])  # Add unvisited neighbors to queue
    return len(visited) == len(adj_list)  # If all nodes are visited, the graph is connected

# Test
disconnected = {'A': ['B'], 'B': ['A'], 'C': []}
print("Is connected?", is_connected(graph))         # True
print("Is connected?", is_connected(disconnected))  # False

# 6. Check for walk/trail/path
def check_walk_type(graph, sequence):
    if not sequence or len(sequence) < 2:
        return "None" # Sequence too short to be a valid walk
    
    is_walk = True # A walk allows repeated nodes and edges
    is_trail = True # A trail allows repeated nodes but not repeated edges
    is_path = True # A path allows no repeated nodes or edges
    edges_seen = set()
    nodes_seen = set()

    for i in range(len(sequence)-1):
        u, v = sequence[i], sequence[i+1]
        if v not in graph.get(u, []):  # If nodes are not adjacent, it's not a valid walk
            return "None"
        if (u, v) in edges_seen or (v, u) in edges_seen:
            is_trail = False  # Edge is repeated
        else:
            edges_seen.add((u, v)) # Store the edge
        if v in nodes_seen:
            is_path = False  # Node is repeated
        nodes_seen.add(u)    # Keep track of visited nodes
    return "Path" if is_path else "Trail" if is_trail else "Walk"

# Test
print("Walk type:", check_walk_type(graph, ['A', 'B', 'A']))  # Trail
print("Walk type:", check_walk_type(graph, ['A', 'C', 'A']))  # Trail
print("Walk type:", check_walk_type(graph, ['A', 'B', 'C']))  # None

# 7. Check if graph is a tree
def is_tree(adj_list):
    return is_connected(adj_list) and not has_cycle(adj_list)  # A tree is connected and has no cycles

def has_cycle(adj_list):
    visited = set()

    def dfs(node, parent):
        visited.add(node)
        for neighbor in adj_list[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):  #Recurse for unvisited neighbors
                    return True
            elif neighbor != parent: #If the neighbor is visited and not parent, cycle exists
                return True
        return False

    start = next(iter(adj_list)) # Start DFS from any one node
    return dfs(start, None)

# Test
tree = {'A': ['B'], 'B': ['A', 'C'], 'C': ['B']}
cycle_graph = {'A': ['B'], 'B': ['A', 'C'], 'C': ['B', 'A']}
print("Is tree?", is_tree(tree))            # True
print("Is tree?", is_tree(cycle_graph))     # False

# 8. Get spanning tree (BFS traversal tree)
def get_spanning_tree(adj_list):
    start = next(iter(adj_list)) # Pick any starting node
    tree = defaultdict(list)
    visited = set()
    queue = deque([start])
    visited.add(start)

    while queue:
        node = queue.popleft()
        for neighbor in adj_list[node]:
            if neighbor not in visited:
                tree[node].append(neighbor) # Add to spanning tree
                tree[neighbor].append(node) # Since undirected, add reverse too
                visited.add(neighbor)
                queue.append(neighbor) # Continue BFS
    return dict(tree)

# Test
print("Spanning Tree:", get_spanning_tree(tree))

# 9. Count leaf nodes in a tree
def count_leaf_nodes(tree):
    return sum(1 for node in tree if len(tree[node]) == 1)  # Leaf = node with only one connection

# Test
print("Leaf count:", count_leaf_nodes(tree))  # Should be 2

# 10. Check if tree is binary
def is_binary_tree(tree):
    return all(len(neighbors) <= 2 for neighbors in tree.values())  # Max 2 connections allowed for binary tree

# Test
print("Is binary tree?", is_binary_tree(tree))  # True

# 11. Find height of a node (max depth of subtree)
def find_height(tree, node):
    def dfs(n, parent):
        heights = [dfs(child, n) for child in tree[n] if child != parent]  # Recurse on children (avoid going back to parent)
        return 1 + max(heights, default=0)  # Height is 1 + max height of children
    return dfs(node, None)

# Test
print("Height from A:", find_height(tree, 'A'))  # Should be 2

# 12. Find depth of a node from root
def find_depth(tree, root, target):
    visited = set()
    queue = deque([(root, 0)]) # Start from root with depth 0

    while queue:
        node, depth = queue.popleft()
        if node == target:
            return depth # Found the target, return its depth
        visited.add(node)
        for neighbor in tree[node]:
            if neighbor not in visited:
                queue.append((neighbor, depth + 1))  # Explore child with incremented depth
    return -1  # Not found

# Test
print("Depth of C from A:", find_depth(tree, 'A', 'C'))  # Should be 2


Sorted Degrees: {'A': 2, 'B': 1, 'C': 1}
Edge List: [('A', 'C'), ('A', 'B')]
Adjacency Matrix: [[0, 1, 1], [1, 0, 0], [1, 0, 0]]
Back to Adjacency List: {'A': ['C', 'B'], 'C': ['A'], 'B': ['A']}
Are A and B adjacent? True
Are B and C adjacent? False
Is complete? True
Is complete? False
Is connected? True
Is connected? False
Walk type: Walk
Walk type: Walk
Walk type: None
Is tree? True
Is tree? False
Spanning Tree: {'A': ['B'], 'B': ['A', 'C'], 'C': ['B']}
Leaf count: 2
Is binary tree? True
Height from A: 3
Depth of C from A: 2
